In [2]:
from PIL import Image
import os, glob
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow import keras

# 데이터 불러온 후 Resize

In [3]:
# 가위, 바위, 보 파일사이즈 변경을 위한 변수 설정
target_size = (28,28)

# 가위, 바위, 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들임
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
images = glob.glob(image_dir_path + "/*.jpg") 
for img in images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
images = glob.glob(image_dir_path + "/*.jpg")  
for img in images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
images= glob.glob(image_dir_path + "/*.jpg")  
for img in images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

# 가위, 바위, 보 데이터 읽는 함수 생성

In [4]:
def load_data(img_path):
    number_of_data = 3600
    img_size = 28
    color = 3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype = np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 0   # 가위 : 0
        idx = idx + 1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 1   # 바위 : 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 2   # 보 : 2
        idx = idx + 1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train) = load_data(image_dir_path)
x_train_norm = x_train / 255.0

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 3600 입니다.
x_train shape: (3600, 28, 28, 3)
y_train shape: (3600,)


# 딥러닝 네트워크 설계

In [12]:
n_channel_1 = 16 # 이미지 특징의 수
n_channel_2 = 16 # 이미지 특징의 수 
n_dense = 32 # 분류기에 사용되는 뉴런의 숫자

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation = 'relu', input_shape=(28,28,3))) # 가위,바위,보 3개
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation = 'relu'))
model.add(keras.layers.Dense(3, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 16)        448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 16)        2320      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                12832     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                

# 딥러닝 네트워크 학습

In [13]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(x_train_norm, y_train, epochs = 10)

train_loss, train_accuracy = model.evaluate(x_train_norm, y_train, verbose=2)
print("train_loss: {} ".format(train_loss))
print("train_accuracy: {}".format(train_accuracy))

Epoch 1/10
113/113 [==============================] - 4s 38ms/step - loss: 1.0350 - accuracy: 0.4561
Epoch 2/10
113/113 [==============================] - 8s 67ms/step - loss: 0.8638 - accuracy: 0.5914
Epoch 3/10
113/113 [==============================] - 4s 37ms/step - loss: 0.7293 - accuracy: 0.6739
Epoch 4/10
113/113 [==============================] - 4s 39ms/step - loss: 0.6172 - accuracy: 0.7386
Epoch 5/10
113/113 [==============================] - 8s 71ms/step - loss: 0.5199 - accuracy: 0.7928
Epoch 6/10
113/113 [==============================] - 8s 68ms/step - loss: 0.4652 - accuracy: 0.8161
Epoch 7/10
113/113 [==============================] - 3s 30ms/step - loss: 0.3848 - accuracy: 0.8539
Epoch 8/10
113/113 [==============================] - 3s 27ms/step - loss: 0.3243 - accuracy: 0.8781
Epoch 9/10
113/113 [==============================] - 5s 41ms/step - loss: 0.2874 - accuracy: 0.8939
Epoch 10/10
113/113 [==============================] - 8s 69ms/step - loss: 0.2659 - accura

# 테스트 데이터 함수 생성

In [14]:
def load_test_data(img_path):
    number_of_data = 300
    img_size = 28
    color = 3
    imgs = np.zeros(number_of_data*img_size*img_size*color,dtype = np.int32).reshape(number_of_data,img_size,img_size,color)
    labels = np.zeros(number_of_data,dtype = np.int32)

    idx = 0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 0   # 가위 : 0
        idx = idx + 1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 1   # 바위 : 1
        idx = idx + 1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype = np.int32)
        imgs[idx,:,:,:] = img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 2   # 보 : 2
        idx = idx + 1
        
    print("테스트데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/sample/rock_scissor_paper" # 테스트데이터 경로 설정
(x_test, y_test) = load_test_data(image_dir_path) 
x_test_norm = x_test/255.0

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

테스트데이터(x_test)의 이미지 개수는 300 입니다.
x_test shape: (300, 28, 28, 3)
y_test shape: (300,)


# 테스트 데이터를 활용한 테스트

In [15]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose = 2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

10/10 - 0s - loss: 0.2266 - accuracy: 0.9300
test_loss: 0.22655975818634033 
test_accuracy: 0.9300000071525574


---

테스트 데이터 만들 때 함수를 기존 함수를 이용하고 거기서 number_of_data만 바꿔서 사용하고 싶은데 잘 적용이 안됐다.   
그래서 새로 만들긴 했는데 어떻게 해야 쓸 수 있는지 공부해봐야겠다.   
배운대로 하긴 했는데 처음이라 그런지 뭘 하는건지 잘 모르겠다.   
공부하면서 훈련용과 테스트용 데이터 분리의 중요성을 알게 됐다.   
epoch을 처음에 20으로 설정했는데 그렇게 하면 테스트데이터에서 accuracy가 1.0으로 나와서 epoch을 10으로 재설정했다.